<a href="https://colab.research.google.com/github/taf9865/Reference-Monitor-Security-Build/blob/master/reference_monitor_taf9856_r2py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/kcg295/repy_v2
%cd /content/repy_v2/scripts
!python3 initialize.py
!python3 build.py
%cd ../RUNNABLE/ 
!wget -O /content/repy_v2/RUNNABLE/myapplication.r2py https://static.us.edusercontent.com/files/cDshGjQKDAvNdu8D4maBH44A
!wget -O /content/repy_v2/RUNNABLE/gettingstartedmonitor.r2py https://static.us.edusercontent.com/files/Gj5dNy8S9d2jui8e1pEulioT
!wget -O /content/repy_v2/RUNNABLE/story.txt https://static.us.edusercontent.com/files/py0yb7fLcIuoRrhMbhW4VRaz
!wget -O /content/repy_v2/RUNNABLE/reference_monitor_netid.r2py https://tinyurl.com/2p8temtu




In [6]:
"""
This security layer inadequately handles A/B storage for files in RepyV2.



Note:
    This security layer uses encasementlib.r2py, restrictions.default, repy.py and Python
    Also you need to give it an application to run.
    python repy.py restrictions.default encasementlib.r2py [security_layer].r2py [attack_program].r2py 
    
    """ 
TYPE="type"
ARGS="args"
RETURN="return"
EXCP="exceptions"
TARGET="target"
FUNC="func"
OBJC="objc"
mycontext['filelock'] = createlock()

class ABFile():
  def __init__(self,filename,create):
    # globals
    mycontext['debug'] = False   
    # local (per object) reference to the underlying file
    self.Afn = filename+'.a'
    self.Bfn = filename+'.b'
    

    # make the files and add 'SE' to the readat file...
    if create:
      self.Afile = ABopenfile(self.Afn,create)
      self.Bfile = ABopenfile(self.Bfn,create)
      self.Afile.writeat('SE',0)
    else:
      self.Afile = ABopenfile(self.Afn,True)
      self.Bfile = ABopenfile(self.Bfn,True)
      #self.Afile.writeat(read_text,0)
      #self.Bfile.writeat(read_text,0)
      
 
  def writeat(self,data,offset):
    
    # Write the requested data to the B file using the sandbox's writeat call
    if offset>=0 and offset <=len(self.Bfile):
      self.Bfile.writeat(data,offset)
    else:
         pass

   
  def readat(self,bytes,offset):
    # Read from the A file using the sandbox's readat...
    return self.Afile.readat(bytes,offset)

  def close(self):
    size = len(self.Bfile)
    if(self.Bfile[0] == 'S' and self.Bfile[size-1] == 'E'):
       removefile(self.Afile)
	     self.Afile = ABopenfile(self.Afn,create)
	     self.Afile.writeat(self.Bfile.readat(None,0),0)
        removefile(self.Bfile)
        self.Afile.close()
        self.Bfile.close()
     else:
	      self.Afile.close()
        self.Bfile.close()



def ABopenfile(filename, create):
  return ABFile(filename,create)




# The code here sets up type checking and variable hiding for you.  You
# should not need to change anything below here.
sec_file_def = {"obj-type":ABFile,
                "name":"ABFile",
                "writeat":{"type":"func","args":(str,(int,long)),"exceptions":Exception,"return":(int,type(None)),"target":ABFile.writeat},
                "readat":{"type":"func","args":((int,long,type(None)),(int,long)),"exceptions":Exception,"return":str,"target":ABFile.readat},
                "close":{"type":"func","args":None,"exceptions":None,"return":(bool,type(None)),"target":ABFile.close}
           }

CHILD_CONTEXT_DEF["ABopenfile"] = {TYPE:OBJC,ARGS:(str,bool),EXCP:Exception,RETURN:sec_file_def,TARGET:ABopenfile}

# Execute the user code
secure_dispatch_module()


TabError: ignored